In [1]:
import pandas as pd
import json

In [2]:
#Sticky end ligation frequency matrix from Potapov et al 2018 
# https://pubmed.ncbi.nlm.nih.gov/30335370/
sticky_end_file = "Sticky_end_ligation_matrix.xlsx"
sticky_end_output_file = "Ligation_fidelity_dictionary.json"

sticky_end_df = pd.read_excel(sticky_end_file, index_col="Overhang")
sticky_end_df.head()

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
Overhang,,,,,,,,,,,,,,,,,,,,,
TTTT,3804,4,96,57,0,0,1,1,5,0,...,0,0,0,0,0,0,1,0,0,0
GTTT,2,5322,3,565,1,44,2,6,0,27,...,0,0,0,0,0,0,0,2,0,0
CTTT,3,2,4742,3,1,0,59,0,0,0,...,0,0,0,0,0,0,0,0,1,0
ATTT,1,3,2,5152,0,0,0,5,0,1,...,0,0,0,0,0,0,0,0,0,0
TGTT,0,0,0,0,5482,11,768,322,5,1,...,0,0,0,0,0,0,0,0,0,0


In [57]:
# Create an empty dictionary to store the results
hf_low = {col: float('nan') for col in sticky_end_df.columns}
hf_medium = {col: float('nan') for col in sticky_end_df.columns}
hf_high = {col: float('nan') for col in sticky_end_df.columns}
hf_ultrahigh = {col: float('nan') for col in sticky_end_df.columns}

print(hf_ultrahigh['AAAA'])

low = 0.8   #determines the threshold to filter misligations, low means will only check those known misligations > 20% of the time
med = 0.9   #medium means will check known misligations that happen >10% of the time
high = 0.99 #high means will check misligations that happen >1% of the time
ultrahigh = 0.9997

# Iterate through columns and find indices where values are greater than 10% of the maximum
for column in sticky_end_df.columns:  # Exclude the 'Overhang' column
    max_value = sticky_end_df[column].max()
    low_threshold = (1 - low) * max_value
    medium_threshold = (1 - med) * max_value
    high_threshold = (1 - high) * max_value
    ultrahigh_threshold = (1 - ultrahigh) * max_value
    max_threshold = 0.9 * max_value
    
    low_indices = sticky_end_df[(sticky_end_df[column] > low_threshold) & (sticky_end_df[column] < max_threshold)].index.tolist()
    medium_indices = sticky_end_df[(sticky_end_df[column] > medium_threshold) & (sticky_end_df[column] < max_threshold)].index.tolist()
    high_indices = sticky_end_df[(sticky_end_df[column] > high_threshold) & (sticky_end_df[column] < max_threshold)].index.tolist()
    ultrahigh_indices = sticky_end_df[(sticky_end_df[column] > ultrahigh_threshold) & (sticky_end_df[column] < max_threshold)].index.tolist()

    hf_low[column] = low_indices
    hf_medium[column] = medium_indices
    hf_high[column] = high_indices
    hf_ultrahigh[column] = ultrahigh_indices

print(hf_ultrahigh['AAAA'])

nan
['GTTT', 'CTTT', 'TTAT', 'TTTG']


In [58]:
combined_dict = {"high": hf_high, "medium": hf_medium, "low": hf_low, "ultrahigh": hf_ultrahigh}

# Write the combined dictionary to the file in JSON format
with open(sticky_end_output_file, "w") as json_file:
    json.dump(combined_dict, json_file)

In [59]:
df = pd.read_json(sticky_end_output_file)
print(df["low"]['GCGT'])

['GCGC']


In [56]:
sticky_end_df.columns[0]

'AAAA'